# 豆瓣用户信息爬虫
#### 目标： 爬取豆瓣用户信息及其关注用户，对其进行简单的数据分析

# 目录
1. 导入库
2. 全局变量
3. 爬虫代码
4. 主函数
5. 程序运行
6. 数据保存

## 1. 导入库

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import time
import pandas as pd

## 2. 一些全局变量

In [2]:
#浏览器头部备选
user_agent = [
    "Mozilla/5.0 (compatible; Baiduspider/2.0; +http://www.baidu.com/search/spider.html)",
    "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; AcooBrowser; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0; Acoo Browser; SLCC1; .NET CLR 2.0.50727; Media Center PC 5.0; .NET CLR 3.0.04506)",
    "Mozilla/4.0 (compatible; MSIE 7.0; AOL 9.5; AOLBuild 4337.35; Windows NT 5.1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
    "Mozilla/5.0 (Windows; U; MSIE 9.0; Windows NT 9.0; en-US)",
    "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Win64; x64; Trident/5.0; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 2.0.50727; Media Center PC 6.0)",
    "Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; WOW64; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 1.0.3705; .NET CLR 1.1.4322)",
    "Mozilla/4.0 (compatible; MSIE 7.0b; Windows NT 5.2; .NET CLR 1.1.4322; .NET CLR 2.0.50727; InfoPath.2; .NET CLR 3.0.04506.30)",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN) AppleWebKit/523.15 (KHTML, like Gecko, Safari/419.3) Arora/0.3 (Change: 287 c9dfb30)",
    "Mozilla/5.0 (X11; U; Linux; en-US) AppleWebKit/527+ (KHTML, like Gecko, Safari/419.3) Arora/0.6",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.8.1.2pre) Gecko/20070215 K-Ninja/2.1.1",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN; rv:1.9) Gecko/20080705 Firefox/3.0 Kapiko/3.0",
    "Mozilla/5.0 (X11; Linux i686; U;) Gecko/20070322 Kazehakase/0.4.5",
    "Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.8) Gecko Fedora/1.9.0.8-1.fc10 Kazehakase/0.5.6",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_3) AppleWebKit/535.20 (KHTML, like Gecko) Chrome/19.0.1036.7 Safari/535.20",
    "Opera/9.80 (Macintosh; Intel Mac OS X 10.6.8; U; fr) Presto/2.9.168 Version/11.52"
]

myAgent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.129 Safari/537.36'
HEADER = {
    'User-Agent':user_agent[6] ,  # 浏览器头部
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',  # 客户端能够接收的内容类型
    'Accept-Language': 'en-US,en;q=0.5',  # 浏览器可接受的语言
    'Connection': 'keep-alive',  # 表示是否需要持久连接
}
#用于登录的cookie信息，查看用户关注需要登录， 之查看用户信息的化无需登录
#cookies = {'cookies':  'bid=K4Nk6OrwACY; __utmz=30149280.1587000963.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); __gads=ID=2024cb86f509fa69:T=1587000965:S=ALNI_Mali8jFClN-pQuMGFkuRp0I-VpWUA; _vwo_uuid_v2=D8035AEDEF679691126366728B2682AD1|a1dd76943d48d1e09072874010a09228; __utmc=30149280; push_noty_num=0; push_doumail_num=0; __utmv=30149280.21622; loc-last-index-location-id="118228"; ll="118228"; __yadk_uid=RGl82ZlXlfRtKpMd4WDmTIoAVLnO8Qpm; ap_v=0,6.0; _pk_ref.100001.8cb4=%5B%22%22%2C%22%22%2C1588668355%2C%22https%3A%2F%2Fmovie.douban.com%2Fsubject%2F26348103%2Fcomments%3Fstart%3D%7B%7D%26limit%3D%7B%7D%26sort%3Dnew_score%26status%3DP%22%5D; _pk_ses.100001.8cb4=*; douban-profile-remind=1; __utma=30149280.926081472.1587000963.1588658977.1588668361.7; __utmt=1; __utmb=30149280.4.10.1588668361; _pk_id.100001.8cb4=985a186b1449e6c8.1587001136.7.1588668868.1588662946.; dbcl2="216228808:Qkhabq9R1fM"'}
#cookies = {'cookie': 'bid=K4Nk6OrwACY; __utmz=30149280.1587000963.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); __gads=ID=2024cb86f509fa69:T=1587000965:S=ALNI_Mali8jFClN-pQuMGFkuRp0I-VpWUA; _vwo_uuid_v2=D8035AEDEF679691126366728B2682AD1|a1dd76943d48d1e09072874010a09228; __utmc=30149280; push_noty_num=0; push_doumail_num=0; __utmv=30149280.21622; loc-last-index-location-id="118228"; ll="118228"; __yadk_uid=RGl82ZlXlfRtKpMd4WDmTIoAVLnO8Qpm; douban-profile-remind=1; _pk_ref.100001.8cb4=%5B%22%22%2C%22%22%2C1588672739%2C%22https%3A%2F%2Fmovie.douban.com%2Fsubject%2F26348103%2Fcomments%3Fstart%3D%7B%7D%26limit%3D%7B%7D%26sort%3Dnew_score%26status%3DP%22%5D; __utma=30149280.926081472.1587000963.1588668361.1588672740.8; _pk_id.100001.8cb4=985a186b1449e6c8.1587001136.8.1588673849.1588669178.; ap_v=0,6.0; dbcl2="216228808:+/kGgJz0Aag"; ck=cZ5X'}

In [3]:
# 一些全局变量

#起始url
BASE_URL = "https://www.douban.com/people/67007418/"
#数据保存在DataFrame中
DATA_NET = pd.DataFrame(columns= ['userId', 'friendId', 'friendUrl'])
DATA_NET = DATA_NET.append([{'userId': '67007418','friendId': '67007418', 'friendUrl': BASE_URL}])

DATA_INFO = pd.DataFrame(columns = ['userId', 'city', 'joinDate'])
#DATA_INFO = DATA_INFO.append([{'userId': '67007418', 'city': '北京', 'joinDate': '2012-12-09'}])

In [4]:
#用于记录爬虫爬到了哪个网页
WEB_FLAG = 0

In [5]:
#用于sleep的随机数
def randomNum(maxNum):
    return np.random.randint(0, maxNum)

## 3. 爬虫具体实现代码

In [6]:
#获取网页Text
def getHTMLtext(url):
    #需登录时添加cookies即可
    #r = requests.get(url, timeout=5, headers=HEADER, cookies = cookies)
    r = requests.get(url, timeout=5, headers=HEADER)
    print("WEB_FLAG:", WEB_FLAG, " status_code: ", r.status_code)
    #         r.raise_for_status()
    #         r.encoding = r.apparent_encoding
    r.encoding = "utf-8"
    return r.text,r.status_code
    

In [7]:
#获取用户信息
def getUserInfo(userId, url):
    #声明全局变量
    global DATA_INFO
    
    try:
        #爬取 用户信息 内容
        htmlText, code =  getHTMLtext(url)
        #使用BeautifulSoup
        soup = BeautifulSoup(htmlText, 'html.parser') 

        #获取用户自身信息， 首先找到 用户信息块
        userInfo = soup.find('div', {'class': 'user-info'})
        '''
        <div class="user-info">

        常居: <a href="https://beijing.douban.com/">北京</a><br>
        <div class="pl">67007418 <br/> 2012-12-09加入</div>
        </br></div>
        '''
        if userInfo is None:
            print(userId, "no user info")
            return 0
        #读取相应信息
        userInfoCity = userInfo.find('a')
        if userInfoCity is None:
            userCity = np.nan
        else:
            userCity = userInfoCity.get_text()
        userDate = userInfo.find('div', {'class': 'pl'}).get_text().split('  ')[-1][:-2]
        #往DataFrame中添加信息
        DATA_INFO = DATA_INFO.append([{'userId': userId, 'city': userCity, 'joinDate': userDate}], ignore_index = True)
        
        return code
    except:
        print("Error 1! WEB FLAG at:", WEB_FLAG)

In [8]:
#获取关注用户信息（ Id 和 Url )
def getFriendUrl(userId, url):
    #声明全局变量
    global DATA_NET
    
    try:
        #爬取 关注用户网页 内容
        htmlText, code =  getHTMLtext(url+'contacts')
        #使用BeautifulSoup
        soup = BeautifulSoup(htmlText, 'html.parser')

        #获取关注用户信息， 首先找到 关注用户块
        friends = soup.findAll(name = 'dl', attrs = {'class' : 'obu'})
        if friends == []:
            print(userId, "have no friends")
        for friend in friends:
            friendTag = BeautifulSoup(str(friend))
            friendNbg = friendTag.findAll(name = 'a', attrs = {'class': 'nbg', "href": re.compile(r"https://www.douban.com/people/(\s\w+)?")})
            friendUrl = friendNbg[0].get('href')
            friendId = friendUrl.split('/')[-2]
            #往DataFrame中添加信息
            DATA_NET = DATA_NET.append([{'userId': userId, 'friendId': friendId, 'friendUrl': friendUrl}], ignore_index = True)
    except:
        print("Error 2! WEB FLAG at:", WEB_FLAG)


In [9]:
#爬虫骨架1: 同时爬取 边和节点信息
def spyder(url = BASE_URL):
    userId = url.split('/')[-2]
    print('userId: ', userId)
    
        #筛选信息添加至DataFrame中
    getUserInfo(userId, url)
        
        #睡眠一会儿
    time.sleep(randomNum(10))
        
        #筛选信息添加至DataFrame中
    getFriendUrl(userId, url)
        
        #睡眠一会儿
    time.sleep(randomNum(10))
    

In [10]:
#爬虫骨架2：爬取DATA_NET中剩余User的Info
def spyderUserInfo(url = BASE_URL):
    
    userId = url.split('/')[-2]
    print('userId: ', userId)
    
        #筛选信息添加至DataFrame中
    code = getUserInfo(userId, url)
        
        #睡眠一会儿
    #time.sleep(randomNum(5))

## 4. 主函数

In [11]:
def main():
    global WEB_FLAG
    while WEB_FLAG < 100000:
        #不爬重复用户
        if DATA_NET.iloc[WEB_FLAG].loc['friendId'] in set(DATA_INFO['userId']):
            WEB_FLAG += 1
            continue

        #从之前爬取的DATA_NET中获取friendUrl 作为下一个目标
        url = DATA_NET.iloc[WEB_FLAG].loc['friendUrl']
        
        #使用骨架1 同时爬取edge和nodeinfo
        #spyder(url)  
        
        #使用骨架2 只爬取nodeinfo
        code = spyderUserInfo(url)
        #if code != 200:
            #print(code)
            #break
        #指示DATA_NET中数据的 flag 下移一位
        WEB_FLAG += 1
    print('Complete')

## 5. 运行主函数

In [292]:
#使用 骨架1 运行爬虫主代码
main()

userId:  yetone
status_code:  200
status_code:  200
userId:  singorcry
status_code:  200
status_code:  200
userId:  yalindongdong
status_code:  200
status_code:  200
userId:  fengjiansun
status_code:  200
status_code:  200
userId:  K.thinking
status_code:  200
status_code:  200
userId:  tclh123
status_code:  200
status_code:  200
userId:  joanna_7
status_code:  200
status_code:  200
userId:  nooriginfever
status_code:  200
status_code:  200
userId:  137701253
status_code:  200
137701253 no user info
status_code:  200
userId:  1233038
status_code:  200
status_code:  200
1233038 have no friends
userId:  DrRay
status_code:  200
status_code:  200
userId:  3458040
status_code:  200
status_code:  200
userId:  137105855
status_code:  200
status_code:  200
userId:  youngker
status_code:  200
status_code:  200
userId:  fadein-out
status_code:  200
status_code:  200
userId:  37746546
status_code:  200
status_code:  200
userId:  musicbug
Error 1! WEB FLAG at: 317
status_code:  200
userId:  234964

status_code:  200
userId:  EUFF
status_code:  200
status_code:  200
userId:  yingliang
status_code:  200
status_code:  200
Error 2! WEB FLAG at: 446
userId:  2464328
status_code:  200
status_code:  200
userId:  xiaosong
status_code:  200
Error 2! WEB FLAG at: 448
userId:  misslalalala
status_code:  200
status_code:  200
userId:  elva_yu
status_code:  200
status_code:  200
userId:  Tirisfal
status_code:  200
status_code:  200
userId:  60768669
status_code:  200


KeyboardInterrupt: 

In [373]:
#使用 骨架2 运行爬取user_info的代码
main()

userId:  Ishtar
WEB_FLAG: 9148  status_code:  200
userId:  ArisAya
WEB_FLAG: 9149  status_code:  200
ArisAya no user info
userId:  mishideai
WEB_FLAG: 9150  status_code:  200
userId:  eleven_777_
WEB_FLAG: 9151  status_code:  200
eleven_777_ no user info
userId:  2118004
WEB_FLAG: 9152  status_code:  200
userId:  moonhouse
WEB_FLAG: 9153  status_code:  200
userId:  littlejenny
WEB_FLAG: 9154  status_code:  200
userId:  zuozuofeng
WEB_FLAG: 9155  status_code:  200
userId:  icehan2046
WEB_FLAG: 9156  status_code:  200
userId:  Yeatsilence
WEB_FLAG: 9157  status_code:  200
userId:  iKelly
WEB_FLAG: 9158  status_code:  200
userId:  lordwong
WEB_FLAG: 9159  status_code:  200
userId:  aiersixiaojie
WEB_FLAG: 9161  status_code:  200
userId:  JuHLa
WEB_FLAG: 9162  status_code:  200
JuHLa no user info
userId:  ariesorange
WEB_FLAG: 9163  status_code:  200
userId:  echocui
WEB_FLAG: 9165  status_code:  200
userId:  mermaid0603
Error 1! WEB FLAG at: 9166
userId:  nielu
WEB_FLAG: 9167  status_code

userId:  jinengjiaohuan
WEB_FLAG: 9382  status_code:  200
userId:  blueskd
Error 1! WEB FLAG at: 9387
userId:  2733685
Error 1! WEB FLAG at: 9389
userId:  tingdog
WEB_FLAG: 9391  status_code:  200
userId:  wenna
WEB_FLAG: 9392  status_code:  200
userId:  theBrave
WEB_FLAG: 9393  status_code:  200
userId:  Fejgl
WEB_FLAG: 9394  status_code:  200
userId:  yisuli
WEB_FLAG: 9395  status_code:  200
userId:  1097461
WEB_FLAG: 9396  status_code:  200
userId:  18768692
WEB_FLAG: 9397  status_code:  200
userId:  38327518
WEB_FLAG: 9398  status_code:  200
userId:  3461680
WEB_FLAG: 9399  status_code:  200
userId:  RDX
WEB_FLAG: 9401  status_code:  200
userId:  adieudusk
WEB_FLAG: 9402  status_code:  200
userId:  arale310
Error 1! WEB FLAG at: 9403
userId:  zhumuyilang
WEB_FLAG: 9404  status_code:  200
userId:  1644460
WEB_FLAG: 9405  status_code:  200
userId:  quatemberkind
WEB_FLAG: 9406  status_code:  200
userId:  maoyingtao
WEB_FLAG: 9407  status_code:  200
userId:  Silvia0712
WEB_FLAG: 9408 

userId:  wanwanwanwanzi
WEB_FLAG: 9594  status_code:  200
userId:  shinewewe
WEB_FLAG: 9595  status_code:  200
userId:  moulder
WEB_FLAG: 9596  status_code:  200
userId:  huang1920
WEB_FLAG: 9597  status_code:  200
userId:  saisaishushu
WEB_FLAG: 9598  status_code:  200
saisaishushu no user info
userId:  62070387
WEB_FLAG: 9599  status_code:  403
62070387 no user info
userId:  lankaiyin
WEB_FLAG: 9600  status_code:  200
userId:  sailaxiaojiong
WEB_FLAG: 9601  status_code:  200
userId:  amang-photo
WEB_FLAG: 9602  status_code:  200
userId:  sixla
WEB_FLAG: 9603  status_code:  200
userId:  26499522
WEB_FLAG: 9604  status_code:  200
userId:  adamwithd
WEB_FLAG: 9605  status_code:  200
adamwithd no user info
userId:  3255070
WEB_FLAG: 9606  status_code:  200
userId:  souppp
WEB_FLAG: 9607  status_code:  200
userId:  sunyitong
WEB_FLAG: 9608  status_code:  403
sunyitong no user info
userId:  liviasugar
WEB_FLAG: 9609  status_code:  200
userId:  suspense7
WEB_FLAG: 9610  status_code:  200
su

userId:  dustbbs
WEB_FLAG: 9775  status_code:  200
userId:  vone
WEB_FLAG: 9776  status_code:  200
userId:  41749081
WEB_FLAG: 9777  status_code:  200
userId:  LoveEricXu
WEB_FLAG: 9778  status_code:  200
userId:  gostcry
WEB_FLAG: 9779  status_code:  200
userId:  3964761
WEB_FLAG: 9780  status_code:  200
userId:  yanzheng
WEB_FLAG: 9781  status_code:  200
userId:  darylhochi
WEB_FLAG: 9782  status_code:  200
userId:  1095161
WEB_FLAG: 9783  status_code:  200
userId:  yejinbi
WEB_FLAG: 9785  status_code:  200
userId:  44295129
WEB_FLAG: 9786  status_code:  200
userId:  juanjuanxingren
WEB_FLAG: 9787  status_code:  200
userId:  xiaxiaokuiguai
WEB_FLAG: 9788  status_code:  200
userId:  cidarlala
WEB_FLAG: 9789  status_code:  200
userId:  gusmo
WEB_FLAG: 9790  status_code:  200
userId:  iamldn
WEB_FLAG: 9791  status_code:  200
userId:  yangchangyi520
WEB_FLAG: 9792  status_code:  200
userId:  zi-buyu
WEB_FLAG: 9795  status_code:  200
userId:  flyhigh
WEB_FLAG: 9796  status_code:  200
user

WEB_FLAG: 9965  status_code:  200
userId:  45054609
WEB_FLAG: 9966  status_code:  200
userId:  Crimma
WEB_FLAG: 9967  status_code:  200
userId:  43861580
WEB_FLAG: 9968  status_code:  200
43861580 no user info
userId:  morningmorning
WEB_FLAG: 9969  status_code:  200
userId:  iCanCan
WEB_FLAG: 9970  status_code:  200
userId:  tutoutou
WEB_FLAG: 9971  status_code:  200
userId:  adai1988
WEB_FLAG: 9973  status_code:  200
userId:  panchito
WEB_FLAG: 9974  status_code:  200
userId:  lucylau
WEB_FLAG: 9975  status_code:  200
userId:  cjeg3220
WEB_FLAG: 9976  status_code:  200
userId:  lengzhiwen
WEB_FLAG: 9979  status_code:  200
userId:  amoona
WEB_FLAG: 9980  status_code:  200
userId:  43517695
WEB_FLAG: 9981  status_code:  200
43517695 no user info
userId:  Pan.da
WEB_FLAG: 9982  status_code:  200
userId:  Rottenmudpit
WEB_FLAG: 9983  status_code:  200
userId:  ccdtg
Error 1! WEB FLAG at: 9984
userId:  mujiman
Error 1! WEB FLAG at: 9985
userId:  runrunlily
WEB_FLAG: 9986  status_code:  20

...

WEB_FLAG: 15066  status_code:  200
userId:  giverny
WEB_FLAG: 15068  status_code:  200
userId:  amigojeff
WEB_FLAG: 15072  status_code:  200
userId:  120053077
WEB_FLAG: 15073  status_code:  200
userId:  iminnow
WEB_FLAG: 15076  status_code:  200
userId:  43973447
WEB_FLAG: 15077  status_code:  200
userId:  yiqing0830
WEB_FLAG: 15082  status_code:  200
userId:  iyuxiao
WEB_FLAG: 15083  status_code:  200
userId:  NO-doubanjiang
WEB_FLAG: 15084  status_code:  200
userId:  VampireEdie
WEB_FLAG: 15085  status_code:  200
userId:  73759477
WEB_FLAG: 15086  status_code:  200
userId:  53834125
WEB_FLAG: 15088  status_code:  200
userId:  yuhengmylove
WEB_FLAG: 15093  status_code:  200
userId:  4698529
WEB_FLAG: 15095  status_code:  200
userId:  NotAndOrXor
WEB_FLAG: 15096  status_code:  200
userId:  65122818
WEB_FLAG: 15097  status_code:  200
userId:  ningmeng710
WEB_FLAG: 15098  status_code:  200
userId:  sangkite
WEB_FLAG: 15099  status_code:  200
userId:  46555584
WEB_FLAG: 15102  status_cod

WEB_FLAG: 15284  status_code:  200
userId:  52408842
Error 1! WEB FLAG at: 15287
userId:  xero.
WEB_FLAG: 15288  status_code:  200
userId:  4657884
WEB_FLAG: 15290  status_code:  200
userId:  31912464
WEB_FLAG: 15291  status_code:  200
userId:  wnwnvnvn
WEB_FLAG: 15294  status_code:  200
userId:  43885833
WEB_FLAG: 15295  status_code:  200
userId:  nrl
WEB_FLAG: 15296  status_code:  200
userId:  2294923
WEB_FLAG: 15297  status_code:  200
2294923 no user info
userId:  wendy0608
WEB_FLAG: 15298  status_code:  200
userId:  72169270
WEB_FLAG: 15299  status_code:  200
userId:  fengjinsheng
WEB_FLAG: 15300  status_code:  200
userId:  shanheshui1986
WEB_FLAG: 15301  status_code:  200
userId:  48324211
WEB_FLAG: 15302  status_code:  200
userId:  sdlee1625-02
WEB_FLAG: 15303  status_code:  200
userId:  lululakehouse
WEB_FLAG: 15305  status_code:  200
userId:  whaletutu
WEB_FLAG: 15306  status_code:  200
userId:  49818266
WEB_FLAG: 15307  status_code:  200
userId:  twinklejune
WEB_FLAG: 15308  s

WEB_FLAG: 15558  status_code:  200
userId:  55507508
WEB_FLAG: 15559  status_code:  200
userId:  19928009
WEB_FLAG: 15560  status_code:  200
userId:  2536300
WEB_FLAG: 15562  status_code:  200
userId:  thunderous
WEB_FLAG: 15563  status_code:  200
userId:  xzf1982
WEB_FLAG: 15564  status_code:  200
userId:  1329593
WEB_FLAG: 15565  status_code:  200
userId:  59881801
WEB_FLAG: 15571  status_code:  200
userId:  Murk
WEB_FLAG: 15572  status_code:  200
userId:  secretjinzhengu
Error 1! WEB FLAG at: 15573
userId:  76184330
WEB_FLAG: 15577  status_code:  200
userId:  Sky4
WEB_FLAG: 15578  status_code:  200
userId:  65202174
WEB_FLAG: 15579  status_code:  200
userId:  bbq2008
WEB_FLAG: 15581  status_code:  200
userId:  bukeguilei
WEB_FLAG: 15583  status_code:  200
userId:  guzhiwang
WEB_FLAG: 15585  status_code:  200
userId:  nemoprincess
WEB_FLAG: 15586  status_code:  200
userId:  shanshu0607
WEB_FLAG: 15591  status_code:  200
userId:  lanjielei
WEB_FLAG: 15594  status_code:  200
userId:  6

WEB_FLAG: 15833  status_code:  403
43677701 no user info
userId:  64799329
WEB_FLAG: 15834  status_code:  403
64799329 no user info
userId:  noyesnoyes
WEB_FLAG: 15836  status_code:  403
noyesnoyes no user info
userId:  vane37
WEB_FLAG: 15837  status_code:  403
vane37 no user info
userId:  2824617
WEB_FLAG: 15838  status_code:  403
2824617 no user info
userId:  33324628
WEB_FLAG: 15839  status_code:  403
33324628 no user info
userId:  zhaoxiaofan
WEB_FLAG: 15840  status_code:  403
zhaoxiaofan no user info
userId:  wenyuanxz
WEB_FLAG: 15841  status_code:  403
wenyuanxz no user info
userId:  52112521
WEB_FLAG: 15843  status_code:  403
52112521 no user info
userId:  youles
WEB_FLAG: 15846  status_code:  403
youles no user info
userId:  woyouzuicaiguai
WEB_FLAG: 15847  status_code:  403
woyouzuicaiguai no user info
userId:  53910218
WEB_FLAG: 15848  status_code:  403
53910218 no user info
userId:  jianghuyiweiyu
WEB_FLAG: 15849  status_code:  403
jianghuyiweiyu no user info
userId:  245522

KeyboardInterrupt: 

In [388]:
DATA_INFO

,userId,city,joinDate
0,67007418,北京,2012-12-09
1,196094934,广东广州,2019-05-07
2,157079038,江苏南京,2017-01-31
3,tjz230,北京,2005-07-18
4,joearde,北京,2005-08-24
...,...,...,...
15556,2159161,北京,2008-01-08
15557,64354416,"Bristol, United Kingdom",2012-09-01
15558,1602355,北京,2007-06-09
15559,55660976,江苏苏州,2011-10-31


In [294]:
DATA_NET

,userId,friendId,friendUrl
0,67007418,67007418,https://www.douban.com/people/67007418/
1,67007418,196094934,https://www.douban.com/people/196094934/
2,67007418,157079038,https://www.douban.com/people/157079038/
3,67007418,tjz230,https://www.douban.com/people/tjz230/
4,67007418,joearde,https://www.douban.com/people/joearde/
...,...,...,...
95391,Tirisfal,kaisic,https://www.douban.com/people/kaisic/
95392,Tirisfal,xuuss,https://www.douban.com/people/xuuss/
95393,Tirisfal,4513498,https://www.douban.com/people/4513498/
95394,Tirisfal,45046918,https://www.douban.com/people/45046918/


## 6. 数据保存至csv文件中：
#### 注意：to_csv中需添加encoding="utf_8_sig"，不然中文字符会乱码

In [298]:
DATA_NET.to_csv('douban_edges.csv', index= False)

In [392]:
#设置encoding="utf_8_sig" 是为了防止中文乱码
DATA_INFO.to_csv('douban_info.csv', index= False, encoding="utf_8_sig")

# Testing Part

In [12]:
htmlText, code = getHTMLtext('https://www.douban.com/people/67007418/')

WEB_FLAG: 0  status_code:  200


In [101]:
userInfo

<div class="user-info">

常居: <a href="https://beijing.douban.com/">北京</a><br>
<div class="pl">67007418 <br/> 2012-12-09加入</div>
</br></div>

如何读取用户信息

用户关注信息

In [180]:
soup = BeautifulSoup(htmlText, 'html.parser')
    
    
    #获取关注用户信息， 首先找到 关注用户块
friends = soup.findAll(name = 'dl', attrs = {'class' : 'obu'})
if friends == []:
        print("need login!")
for friend in friends:
    friendTag = BeautifulSoup(str(friend))
    friendNbg = friendTag.findAll(name = 'a', attrs = {'class': 'nbg', "href": re.compile(r"https://www.douban.com/people/(\s\w+)?")})
    friendUrl = friendNbg[0].get('href')
    friendId = friendUrl.split('/')[-2]

In [181]:
friendId

'36479321'

In [169]:
friendUrl

'https://www.douban.com/people/36479321/'

In [391]:
#第一次IP被封禁
#WEB_FLAG = 300

#第二次IP被封禁
#WEB_FLAG = 452

#第三次被封禁
#WEB_FLAG = 1314

#第四次IP被封禁
#WEB_FLAG = 2172

#五
#WEB_FLAG = 3459
#六
#WEB_FLAG = 4031

#WEB_FLAG = 5585

#WEB_FLAG = 6334

#WEB_FLAG =7077

#WEB_FLAG = 9148

#WEB_FLAG = 15788

#WEB_FLAG = 16942